# 1. Packages to Import

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from itertools import combinations

print("Packages imported successfully!")


# 2. Load Dataset

In [ ]:

file_path = r'D:/IAAC/Classes/AI Programming for Design/Submission/Building_Permits_S.csv'
df = pd.read_csv(file_path, low_memory=False)

print("Dataset loaded successfully!")
print(df.head())


# 3. Preprocessing

In [ ]:

df['Days_to_Issue'] = pd.to_numeric(df['Days_to_Issue'], errors='coerce')
df = df.dropna(subset=['Days_to_Issue'])
features = ['Supervisor District', 'Zipcode', 'Estimated Cost', 'Revised Cost', 'Proposed Units']
df = df.dropna(subset=features)

for col in features:
    if df[col].dtype == 'object':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))

print("Preprocessing completed successfully!")
print(df.head())


# 4. Model Evaluation

In [ ]:

def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    return mse, r2

print("Model evaluation function defined successfully!")


# 5. Applying Models

In [ ]:

all_results = []
for r in range(1, len(features) + 1):
    for combo in combinations(features, r):
        X = df[list(combo)]
        y = df['Days_to_Issue']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        lin_reg = LinearRegression()
        lin_reg.fit(X_train, y_train)
        lin_mse, lin_r2 = evaluate_model(lin_reg, X_test, y_test)
        all_results.append(['Linear Regression', combo, lin_mse, lin_r2])

        ridge_reg = Ridge(alpha=1.0)
        ridge_reg.fit(X_train, y_train)
        ridge_mse, ridge_r2 = evaluate_model(ridge_reg, X_test, y_test)
        all_results.append(['Ridge Regression', combo, ridge_mse, ridge_r2])

        lasso_reg = Lasso(alpha=0.1)
        lasso_reg.fit(X_train, y_train)
        lasso_mse, lasso_r2 = evaluate_model(lasso_reg, X_test, y_test)
        all_results.append(['Lasso Regression', combo, lasso_mse, lasso_r2])

        poly = PolynomialFeatures(degree=2)
        X_train_poly = poly.fit_transform(X_train)
        X_test_poly = poly.transform(X_test)
        poly_reg = LinearRegression()
        poly_reg.fit(X_train_poly, y_train)
        poly_mse, poly_r2 = evaluate_model(poly_reg, X_test_poly, y_test)
        all_results.append(['Polynomial Regression', combo, poly_mse, poly_r2])

        tree_reg = DecisionTreeRegressor(max_depth=5, random_state=42)
        tree_reg.fit(X_train, y_train)
        tree_mse, tree_r2 = evaluate_model(tree_reg, X_test, y_test)
        all_results.append(['Decision Tree', combo, tree_mse, tree_r2])

print("Models applied successfully!")


# 6. Model Evaluation Results

In [ ]:

results_df = pd.DataFrame(all_results, columns=['Model', 'Features', 'MSE', 'R2'])
best_results_per_model = results_df.loc[results_df.groupby('Model')['R2'].idxmax()]
pd.set_option('display.max_colwidth', None)

print("Best Results per Model:")
print(best_results_per_model)

# Print the best model and feature combination
best_overall = best_results_per_model.loc[best_results_per_model['R2'].idxmax()]
print(f"Best Overall Model: {best_overall['Model']}")
print(f"Best Feature Combination: {list(best_overall['Features'])}")
print(f"R² Score: {best_overall['R2']}")


# 7. Visualization

In [ ]:

top_results = results_df.sort_values(by='R2', ascending=False).head(5)
plt.figure(figsize=(12, 8))
plt.barh([f"{row['Model']} {list(row['Features'])}" for _, row in top_results.iterrows()], top_results['R2'], color='teal')
plt.xlabel('R² Score')
plt.ylabel('Model and Features')
plt.title('Top 5 Model Combinations - R² Scores')
plt.tight_layout()
plt.show()

print("Visualization completed successfully!")
